In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [9]:
N = int(input())
A = list(map(int, input().split()))

MAX = max(A) + 1

size = 1
while size < MAX:
    size *= 2
    
node = [0] * size * 2

# pos: 0-indexed
def update(pos, x):
    pos += size
    node[pos] = max(node[pos], x)
    while pos >= 2:
        pos //= 2
        node[pos] = max(node[pos*2], node[pos*2+1])
    
def query(l, r, a, b, n):
    if r <= a or b <= l:
        return 0
    if l <= a and b <= r:
        return node[n]
    
    mid = (a + b) // 2
    vl = query(l, r, a, mid, n*2)
    vr = query(l, r, mid, b, n*2+1)
    
    return max(vl, vr)

ans = 0
for i in range(N-1, -1, -1):
    x = A[i]
    y = query(x+1, size, 0, size, 1)
    update(x, y + 1)
    ans = max(ans, y+1)
print(ans)

1
